<a href="https://colab.research.google.com/github/bartoszpankratz/VAR_neural_network/blob/master/PREOPTIMIZER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def preOptimizationSearch(model, train_loader, numSim, alpha):
    loss = []
    data, target = train_loader.dataset.tensors[0], train_loader.dataset.tensors[1]
    output = model(data)
    weights_0 = model.rnn.all_weights[0][0].data
    weights_1 = model.rnn.all_weights[0][1].data
    weights_2 = model.rnn.all_weights[0][2].data
    weights_3 = model.rnn.all_weights[0][3].data
    weights_4 = model.reg.weight.data
    loss_best = quantile_regression_dk(alpha, target, output)

    start = time.time()
    for j in range(numSim):
        xShape, yShape = model.rnn.all_weights[0][0].shape
        model.rnn.all_weights[0][0].data = torch.tensor(np.random.rand(xShape, yShape)*2 - 1)
        model.rnn.all_weights[0][1].data = torch.tensor(np.random.rand(xShape, 6)*2 - 1)
        model.rnn.all_weights[0][2].data = torch.tensor(np.random.rand(xShape)*2 - 1)
        model.rnn.all_weights[0][3].data = torch.tensor(np.random.rand(xShape)*2 - 1)
        xShape, yShape = model.reg.weight.data.shape
        model.reg.weight.data = torch.tensor(np.random.rand(xShape, yShape)*2-1)
        output = model(data)
        loss.append(quantile_regression_dk(alpha, target, output))
        if loss[-1] < loss_best:
            loss_best = loss[-1]
            weights_0 = model.rnn.all_weights[0][0].data
            weights_1 = model.rnn.all_weights[0][1].data
            weights_2 = model.rnn.all_weights[0][2].data
            weights_3 = model.rnn.all_weights[0][3].data
            weights_4 = model.reg.weight.data
    end = time.time()
    model.rnn.all_weights[0][0].data = weights_0
    model.rnn.all_weights[0][1].data = weights_1
    model.rnn.all_weights[0][2].data = weights_2
    model.rnn.all_weights[0][3].data = weights_3
    model.reg.weight.data = weights_4
    return(model, loss)
    #end - start